In [1]:
import os
from yaulab_processing import get_filePaths_to_extract_prepro, createNWB_prepro, lfp_params_spikeInterface_default

In [ ]:
###############################################################################################################################################################
# RAW PATH/FILE INFORMATION:
# If "fileName = None" it will search for all the YAML files within the "parentFolder_rawNWB" that has NEV & NWB in the same folder
###############################################################################################################################################################
parentFolder_rawNWB = 'Y:\Data_Albus\Albus_NeuralData\Albus-S1'
fileName = None # None or example of sessionName = 'exp2024-04-18-122626'

###############################################################################################################################################################
# PREPRO PARENT FOLDER INFORMATION:
# CAREFUL on the parent paths¡¡ It will create the same tree folder path to match the RAW file tree path relative its "parentFolder_raw"
###############################################################################################################################################################
parentFolder_preproNWB = "Y:\\Data_Albus\\Albus_NeuralData\\Albus-S1_preproNWB"

###############################################################################################################################################################
# Parent folder where to search for expDAY_log.XLSX files that contain Receptive Field Information
# For S1 recordings in particular, it is assume that at this point this information MUST exists and added at this stage
# Set to "None" to not search for this information. 
###############################################################################################################################################################
expLog_parentFolder = "Y:\\Data_Albus\\Albus_NeuralData\\Albus-S1_logs_xlsx"

# If True. It will check that all the electrodes/probes and field-names are completely filled, and print a message if something is missing
# If False.  It will throw an error if something is missing
skipMissing_RF = False # Default = False

# Update Receptive field values in case the information already exists in the raw NWB's Electrode Table
updateRF = True # Default = True

In [ ]:
###############################################################################################################################################################
# To create a *_prepro.NWB file from Original "raw" NWB the following steps will be performed:

###############################################################################################################################################################
# 1) Create LFPs : low-pass Filter of the signal coming from all the aquisition groups (Electrode/Probes). You need to provide the lfp filter & sampling params:
#                  This variable can be loaded from yaulab_processing with the following default values:
#       lfp_params_spikeInterface_default = {
#           'lfp_description': 'SpikeInterface Preprocessing: 
#                               1) BANDPASS (see scipy.signal.iirfilter): 
#                                       Butterworth filter using second-order sections. 
#                                       LowFreq: (order:5, freq-corner:0.1); 
#                                       HighFreq: (order:5, freq-corner:300.0). 
#                                       Margin ms: 5.0. 
#                                       Direction: forward-backward. 
#                               2) Resample (see scipy.signal.resample): Sampling rate 1KHz',
#           'bandpass': {
#               'freq_min': 0.1,
#               'freq_max': 300.0,
#               'margin_ms': 5.0,
#               'dtype': None,
#               'coeff': None,
#               'add_reflect_padding': False,
#               'filter_order': 5,
#               'filter_mode': 'sos',
#               'ftype': 'butter',
#               'direction': 'forward-backward'
#           },
#           'resample': {
#                 'resample_rate': 1000,
#                 'margin_ms': 100.0,
#                 'dtype': None,
#                 'skip_checks': False,
#           }
#         }
#
lfp_params_spikeInterface = lfp_params_spikeInterface_default

###############################################################################################################################################################
#  You can choose to remove or not the original RAW data. Default = True
#
removeRipple_RawEphys = True


###############################################################################################################################################################
# 2) Remove Analog stim that are no longer needed or will be downsampled: 
#    Possible Stimuli to remove and/or Resample: ['fixON', 'visualON', 'rewardON', 'leftAccelerometer', 'leftCommand', 'rightAccelerometer', 'rightCommand', 'thermistors']
#    Define a list of the stimuli to remove. Accelerometers are kept at the original sampling rate to avoid aliasing
#    Default = ['fixON', 'visualON', 'rewardON', 'leftCommand', 'rightCommand', 'thermistors']
#
removeRipple_stimulus_list = ['fixON', 'visualON', 'rewardON', 'leftCommand', 'rightCommand', 'thermistors']

###############################################################################################################################################################
# 3) Downsampling Analog Stim containers: It will save downsampled Ripple-analog stim channels:
#    Create a list of dictionaries with the following format:
#    stimResample_params = {
#              stim1_NAME: {
#                           'use_spikeInterface': False, # True or False 
#                           'resample': {'resample_rate': 1000, 'margin_ms': 100.0, 'dtype': None, 'skip_checks': False} # LFP-Resampling parameters 
#              },
#              stim2_NAME: {
#                           'use_spikeInterface': False, # True or False 
#                           'resample': {'resample_rate': 1000, 'margin_ms': 100.0, 'dtype': None, 'skip_checks': False} # LFP-Resampling parameters 
#              },
#              keep adding stim to resample
#       }
#
#   Default: ONLY "thermistors" will be downsampled
#
stimResample_params = {
    'thermistors': {
        'use_spikeInterface': False,
        'resample': lfp_params_spikeInterface_default['resample']
    }
}

###############################################################################################################################################################
# 4) Remove Behavior-containers no longer needed or that will be downsampled: Because behavior module has container-specific names and attributes, they will 
#               need direct/specific call to be removed
#
removeRipple_FootBar = True

#
#   Ripple eye tracking and pupil-diameter will be removed only if they are going to be downsampled. This will be defined by the variable "eyeResample_params":
#   If eyeResample_params = None it will keep original Ripple eye data. Otherwise should be a dictionary with the following format:
#   Default = { 'use_spikeInterface': False, # True or False 
#               'resample': {'resample_rate': 1000, 'margin_ms': 100.0, 'dtype': None, 'skip_checks': False} # LFP-Resampling parameters
#            }
#
eyeResample_params = {
    'use_spikeInterface': False, 
    'resample': lfp_params_spikeInterface_default['resample']
}

In [ ]:
###############################################################################################################################################################
# Get the path names of raw-NWB. To be a valid raw-NWB, its YAML file must be on the same folder as the NWB and no sufixes added to the NWB file name
###############################################################################################################################################################
# get raw NWB file Paths with potential ephys data. 
# It will check that the YAML file is in the same folder as the raw-NWB (need it to check some receptive field params)
nwbFiles2convert = get_filePaths_to_extract_prepro(parentFolder=parentFolder_rawNWB, fileName=fileName)  

del fileName

print('You will convert {} files'.format(len(nwbFiles2convert)))


In [ ]:
print('These are the RAW filePaths that are going to be converted:')
for nwbRaw_path in nwbFiles2convert:
    print(nwbRaw_path)

In [ ]:
##########################################################################################################
#                                       CREATE NWB preprocessed
##########################################################################################################  

for nwbRaw_path in nwbFiles2convert:

    ##################################################################################################################################################### 
    # CHECK FOLDERS

    # Replace the "RAW" parentFolder by the new "PREPRO" parentFolder
    nwbPrepro_parentFolder, nwbFile_name = os.path.split(nwbRaw_path.replace(parentFolder_rawNWB, parentFolder_preproNWB))

    # Check if the immediate container folder has the same name as the NWB file itself (Standar format to contain RAW files: YAML, NEV, NS5, EYE)
    # If so, remove that folder to only mirrow the remaining tree path (i.e., start to decompress the paths) 
    fileName, _ = os.path.splitext(nwbFile_name)
    
    if nwbPrepro_parentFolder.endswith(fileName):
        nwbPrepro_parentFolder, _ = os.path.split(nwbPrepro_parentFolder)
    
    print('Processing file: {} ............\n\trawPath: {}'.format(fileName, nwbRaw_path))
    if not os.path.isdir(nwbPrepro_parentFolder):
        print('Creating prepro folder : ', nwbPrepro_parentFolder, '\n')
        os.makedirs(nwbPrepro_parentFolder) # Do the entire tree
    
    ###############################################################################################################################################################
    # If the file already exists, Don't do anything. 
    if os.path.isfile(os.path.join(nwbPrepro_parentFolder, fileName + '_prepro.nwb')):

        print('\nPreprocessed file ALREADY EXISTS!!!!\n\tPrepro Path: {}\n\n'.format(os.path.join(nwbPrepro_parentFolder, fileName + '_prepro.nwb')))

    else:

        createNWB_prepro(
            nwbRaw_path = nwbRaw_path,
            nwbPrepro_parentFolder = nwbPrepro_parentFolder,
            removeRipple_RawEphys = removeRipple_RawEphys, 
            removeRipple_FootBar = removeRipple_FootBar,
            removeRipple_stimulus_list = removeRipple_stimulus_list, 
            lfp_params_spikeInterface = lfp_params_spikeInterface,
            eyeResample_params = eyeResample_params,
            stimResample_params = stimResample_params,
            expLog_parentFolder = expLog_parentFolder,
            updateRF = updateRF, 
            skipMissing_RF = skipMissing_RF,
            verbose=True
            )